In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import json
import seaborn as sns
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [4]:
# train_solutions = json.loads("/kaggle/input/arc-prize-2025/arc-agi_training_solutions.json")
# train_challenges = json.loads("/kaggle/input/arc-prize-2025/arc-agi_training_challenges.json")
# eval_solutions = json.loads("/kaggle/input/arc-prize-2025/arc-agi_evaluation_solutions.json")
# eval_challenges = json.loads("/kaggle/input/arc-prize-2025/arc-agi_evaluation_challenges.json")

# file_path
train_solutions = json.loads(open("data/ARC_Prize_2025/arc-agi_training_solutions.json").read())
train_challenges = json.loads(open("data/ARC_Prize_2025/arc-agi_training_challenges.json").read())
eval_solutions = json.loads(open("data/ARC_Prize_2025/arc-agi_evaluation_solutions.json").read())
eval_challenges = json.loads(open("data/ARC_Prize_2025/arc-agi_evaluation_challenges.json").read())

In [5]:
train_solutions

{'00576224': [[[3, 2, 3, 2, 3, 2],
   [7, 8, 7, 8, 7, 8],
   [2, 3, 2, 3, 2, 3],
   [8, 7, 8, 7, 8, 7],
   [3, 2, 3, 2, 3, 2],
   [7, 8, 7, 8, 7, 8]]],
 '007bbfb7': [[[7, 0, 7, 0, 0, 0, 7, 0, 7],
   [7, 0, 7, 0, 0, 0, 7, 0, 7],
   [7, 7, 0, 0, 0, 0, 7, 7, 0],
   [7, 0, 7, 0, 0, 0, 7, 0, 7],
   [7, 0, 7, 0, 0, 0, 7, 0, 7],
   [7, 7, 0, 0, 0, 0, 7, 7, 0],
   [7, 0, 7, 7, 0, 7, 0, 0, 0],
   [7, 0, 7, 7, 0, 7, 0, 0, 0],
   [7, 7, 0, 7, 7, 0, 0, 0, 0]]],
 '009d5c81': [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 7, 7, 7, 7, 7, 7, 7, 7, 7],
   [0, 0, 0, 0, 0, 7, 0, 0, 0, 7, 0, 7, 0, 7],
   [0, 0, 0, 0, 0, 7, 0, 7, 0, 7, 0, 0, 0, 7],
   [0, 0, 0, 0, 0, 7, 7, 7, 7, 7, 7, 7, 7, 7],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0

### Neural Networks

In [6]:
device = "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [8]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [9]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([5], device='cuda:0')
